In [10]:
import numpy as np
from numpy import mean
from numpy import std
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
import xgboost
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, BatchNormalization, Dropout
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import optimizers
from keras import callbacks
from keras.models import model_from_json



SyntaxError: invalid syntax (1123631714.py, line 34)

In [3]:
df = pd.read_excel(r'/Users/sayanbiswas/Downloads/Final ML CSDH dataset.xlsx')
df.head()

,Registrar Taking Referral,Consultant,Age,Sex,Headache,Dementia,GCS,Motor weakness,Midline Shift,CSDH size,QoL,Anticoagulation,Acceptance Status
0,Gill Sharples,Karabatsou K,87,0,0,0,15,0,0,1,0,1,0
1,Pepple,D'Urso P,87,1,0,1,11,1,0,1,1,0,0
2,Vyziotis,Evans J,88,1,0,1,14,0,0,2,1,0,0
3,Pepple,Evans J,75,1,0,1,14,0,0,1,1,1,0
4,Vyziotis,Evans J,79,1,0,0,15,0,0,1,0,1,0


In [4]:
df = df.sample(frac = 1)

X = df.drop(['Registrar Taking Referral', 'Consultant'], axis = 1)

In [5]:
tf.random.set_seed(13)
tf.debugging.set_log_device_placement(False)

#Make train set and temporary test set - 11 columns
train_dataset, temp_test_dataset = train_test_split(X, test_size = 0.4)

#Make test set and validation set - 11 columns
test_dataset, valid_dataset = train_test_split(temp_test_dataset, test_size = 0.5)

#Outcome variable columns - 10 columns
train_labels = train_dataset.pop('Acceptance Status')
test_labels = test_dataset.pop('Acceptance Status')
valid_labels = valid_dataset.pop('Acceptance Status')

print(train_labels.shape)
print(train_dataset.shape)
print(test_dataset.shape)

train_stats = train_dataset.describe()
train_stats = train_stats.transpose()

(298,)
(298, 10)
(100, 10)


In [6]:
#Normalizing the predictor variables
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)
normed_valid_dataset = norm(valid_dataset)

In [7]:
#Creating and compiling the model
def build_model2_three_hiden_layers():
  model = Sequential()

  model.add(Dense(32, input_shape = (normed_train_data.shape[1],)))
  model.add(Dense(32, Activation('relu')))
  model.add(Dense(64, Activation('relu')))
  model.add(Dense(128, Activation('relu')))
  model.add(Dense(1))

  learning_rate = 0.001
  optimizer = optimizers.SGD(learning_rate)
  model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits=True), optimizer = optimizer, metrics = ['accuracy'])

  return model

model = build_model2_three_hiden_layers()

2021-12-10 03:08:22.921954: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
#Fitting the model

earlystopping = callbacks.EarlyStopping(monitor ="val_loss", 
                                        mode ="min", patience = 5, 
                                        restore_best_weights = True)

model.fit(
    normed_train_data,
    train_labels,
    batch_size = 16,
    epochs = 400,
    verbose = 1,
    shuffle = True,
    steps_per_epoch = int(normed_train_data.shape[0] / 16) ,
    validation_data = (normed_valid_dataset, valid_labels),
    callbacks = [earlystopping],
)



Epoch 1/400
18/18 [==============================] - 1s 10ms/step - loss: 0.7224 - accuracy: 0.4826 - val_loss: 0.7249 - val_accuracy: 0.5800
Epoch 2/400
18/18 [==============================] - 0s 2ms/step - loss: 0.7170 - accuracy: 0.4787 - val_loss: 0.7205 - val_accuracy: 0.5800
Epoch 3/400
18/18 [==============================] - 0s 3ms/step - loss: 0.7127 - accuracy: 0.4574 - val_loss: 0.7163 - val_accuracy: 0.5800
Epoch 4/400
18/18 [==============================] - 0s 3ms/step - loss: 0.7094 - accuracy: 0.4965 - val_loss: 0.7116 - val_accuracy: 0.5800
Epoch 5/400
18/18 [==============================] - 0s 3ms/step - loss: 0.7011 - accuracy: 0.4752 - val_loss: 0.7077 - val_accuracy: 0.5800
Epoch 6/400
18/18 [==============================] - 0s 3ms/step - loss: 0.7026 - accuracy: 0.4823 - val_loss: 0.7034 - val_accuracy: 0.5800
Epoch 7/400
18/18 [==============================] - 0s 3ms/step - loss: 0.6950 - accuracy: 0.5106 - val_loss: 0.6989 - val_accuracy: 0.5800
Epoch 8/400


Epoch 59/400
18/18 [==============================] - 0s 2ms/step - loss: 0.4941 - accuracy: 0.7624 - val_loss: 0.4968 - val_accuracy: 0.8100
Epoch 60/400
18/18 [==============================] - 0s 3ms/step - loss: 0.4993 - accuracy: 0.7660 - val_loss: 0.4926 - val_accuracy: 0.8100
Epoch 61/400
18/18 [==============================] - 0s 2ms/step - loss: 0.4876 - accuracy: 0.7553 - val_loss: 0.4883 - val_accuracy: 0.8100
Epoch 62/400
18/18 [==============================] - 0s 2ms/step - loss: 0.4888 - accuracy: 0.7766 - val_loss: 0.4840 - val_accuracy: 0.8100
Epoch 63/400
18/18 [==============================] - 0s 2ms/step - loss: 0.4950 - accuracy: 0.7411 - val_loss: 0.4799 - val_accuracy: 0.8300
Epoch 64/400
18/18 [==============================] - 0s 2ms/step - loss: 0.4599 - accuracy: 0.8121 - val_loss: 0.4753 - val_accuracy: 0.8300
Epoch 65/400
18/18 [==============================] - 0s 2ms/step - loss: 0.4769 - accuracy: 0.7837 - val_loss: 0.4710 - val_accuracy: 0.8500
Epoch 

18/18 [==============================] - 0s 3ms/step - loss: 0.2759 - accuracy: 0.9078 - val_loss: 0.2732 - val_accuracy: 0.9300
Epoch 117/400
18/18 [==============================] - 0s 3ms/step - loss: 0.2710 - accuracy: 0.9043 - val_loss: 0.2703 - val_accuracy: 0.9300
Epoch 118/400
18/18 [==============================] - 0s 3ms/step - loss: 0.2842 - accuracy: 0.8865 - val_loss: 0.2674 - val_accuracy: 0.9300
Epoch 119/400
18/18 [==============================] - 0s 3ms/step - loss: 0.2699 - accuracy: 0.9043 - val_loss: 0.2642 - val_accuracy: 0.9300
Epoch 120/400
18/18 [==============================] - 0s 2ms/step - loss: 0.2505 - accuracy: 0.9291 - val_loss: 0.2612 - val_accuracy: 0.9300
Epoch 121/400
18/18 [==============================] - 0s 2ms/step - loss: 0.2611 - accuracy: 0.9113 - val_loss: 0.2586 - val_accuracy: 0.9300
Epoch 122/400
18/18 [==============================] - 0s 3ms/step - loss: 0.2571 - accuracy: 0.8936 - val_loss: 0.2555 - val_accuracy: 0.9300
Epoch 123/400

18/18 [==============================] - 0s 2ms/step - loss: 0.1600 - accuracy: 0.9574 - val_loss: 0.1570 - val_accuracy: 0.9700
Epoch 174/400
18/18 [==============================] - 0s 2ms/step - loss: 0.1629 - accuracy: 0.9574 - val_loss: 0.1556 - val_accuracy: 0.9700
Epoch 175/400
18/18 [==============================] - 0s 2ms/step - loss: 0.1619 - accuracy: 0.9645 - val_loss: 0.1543 - val_accuracy: 0.9700
Epoch 176/400
18/18 [==============================] - 0s 2ms/step - loss: 0.1652 - accuracy: 0.9468 - val_loss: 0.1532 - val_accuracy: 0.9700
Epoch 177/400
18/18 [==============================] - 0s 2ms/step - loss: 0.1520 - accuracy: 0.9504 - val_loss: 0.1520 - val_accuracy: 0.9700
Epoch 178/400
18/18 [==============================] - 0s 2ms/step - loss: 0.1596 - accuracy: 0.9504 - val_loss: 0.1508 - val_accuracy: 0.9700
Epoch 179/400
18/18 [==============================] - 0s 3ms/step - loss: 0.1629 - accuracy: 0.9716 - val_loss: 0.1500 - val_accuracy: 0.9700
Epoch 180/400

18/18 [==============================] - 0s 2ms/step - loss: 0.1077 - accuracy: 0.9716 - val_loss: 0.1109 - val_accuracy: 0.9700
Epoch 231/400
18/18 [==============================] - 0s 3ms/step - loss: 0.1337 - accuracy: 0.9504 - val_loss: 0.1107 - val_accuracy: 0.9700
Epoch 232/400
18/18 [==============================] - 0s 2ms/step - loss: 0.1159 - accuracy: 0.9681 - val_loss: 0.1102 - val_accuracy: 0.9700
Epoch 233/400
18/18 [==============================] - 0s 2ms/step - loss: 0.1103 - accuracy: 0.9610 - val_loss: 0.1096 - val_accuracy: 0.9700
Epoch 234/400
18/18 [==============================] - 0s 2ms/step - loss: 0.1095 - accuracy: 0.9752 - val_loss: 0.1091 - val_accuracy: 0.9700
Epoch 235/400
18/18 [==============================] - 0s 2ms/step - loss: 0.1302 - accuracy: 0.9504 - val_loss: 0.1087 - val_accuracy: 0.9700
Epoch 236/400
18/18 [==============================] - 0s 2ms/step - loss: 0.1066 - accuracy: 0.9610 - val_loss: 0.1084 - val_accuracy: 0.9700
Epoch 237/400

18/18 [==============================] - 0s 3ms/step - loss: 0.0985 - accuracy: 0.9716 - val_loss: 0.0925 - val_accuracy: 0.9900
Epoch 288/400
18/18 [==============================] - 0s 2ms/step - loss: 0.0974 - accuracy: 0.9681 - val_loss: 0.0923 - val_accuracy: 0.9900
Epoch 289/400
18/18 [==============================] - 0s 2ms/step - loss: 0.0919 - accuracy: 0.9716 - val_loss: 0.0922 - val_accuracy: 0.9900
Epoch 290/400
18/18 [==============================] - 0s 3ms/step - loss: 0.0994 - accuracy: 0.9716 - val_loss: 0.0921 - val_accuracy: 0.9900
Epoch 291/400
18/18 [==============================] - 0s 2ms/step - loss: 0.0984 - accuracy: 0.9610 - val_loss: 0.0920 - val_accuracy: 0.9900
Epoch 292/400
18/18 [==============================] - 0s 3ms/step - loss: 0.0996 - accuracy: 0.9716 - val_loss: 0.0917 - val_accuracy: 0.9900
Epoch 293/400
18/18 [==============================] - 0s 2ms/step - loss: 0.0824 - accuracy: 0.9858 - val_loss: 0.0912 - val_accuracy: 0.9900
Epoch 294/400

18/18 [==============================] - 0s 3ms/step - loss: 0.0861 - accuracy: 0.9787 - val_loss: 0.0864 - val_accuracy: 0.9900
Epoch 345/400
18/18 [==============================] - 0s 3ms/step - loss: 0.0844 - accuracy: 0.9823 - val_loss: 0.0861 - val_accuracy: 0.9900
Epoch 346/400
18/18 [==============================] - 0s 3ms/step - loss: 0.0958 - accuracy: 0.9787 - val_loss: 0.0863 - val_accuracy: 0.9900
Epoch 347/400
18/18 [==============================] - 0s 3ms/step - loss: 0.0873 - accuracy: 0.9787 - val_loss: 0.0862 - val_accuracy: 0.9900
Epoch 348/400
18/18 [==============================] - 0s 3ms/step - loss: 0.0732 - accuracy: 0.9823 - val_loss: 0.0861 - val_accuracy: 0.9900
Epoch 349/400
18/18 [==============================] - 0s 3ms/step - loss: 0.0825 - accuracy: 0.9858 - val_loss: 0.0858 - val_accuracy: 0.9900
Epoch 350/400
18/18 [==============================] - 0s 3ms/step - loss: 0.1058 - accuracy: 0.9716 - val_loss: 0.0862 - val_accuracy: 0.9900
Epoch 351/400

In [14]:
#Plotting the accuracy for train and validation sets for 100 epcohs
plt.plot(model['accuracy'])
plt.plot(model['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Cross-Validation'], loc = 'upper left')
plt.show()

TypeError: 'Sequential' object is not subscriptable